In [1]:
using Preferences, BenchmarkTools, Plots
using LinearAlgebra, PreallocationTools, ForwardDiff, DiffResults
using MPI
include("XBraid.jl")
using .XBraid

MPI.Init()
comm = MPI.COMM_WORLD

MPI.Comm(Ptr{Nothing} @0x00007fd9d4d74c20)

In [14]:
# system parameters (globally scoped)
const nₓ = 16
const Δx = 2π / nₓ
const Δt = 0.1
const basis_rank = 2
x = Array(range(0.0, 2π - Δx, nₓ))

# preallocations (performance critical, passed as arguments)
x_new = zeros(nₓ)
y_new = zeros(nₓ)

struct burger_app
    x
    # preallocated caches that support ForwardDiff:
    x_d::DiffCache
    y_d::DiffCache
end

burger = burger_app(
    x,
    DiffCache(x_new),
    DiffCache(y_new)
);

In [15]:
function interp_periodic(coord, f)
    bound = nₓ
    scaled = coord / Δx
    i = trunc(scaled)
    r = scaled - i
    i = mod1(Int(i + 1), bound)
    (1 - r) * f[i] + r * f[mod1(i + 1, bound)]
end

function semi_lagrangian(burger::burger_app, y, Δt)
    x_back = get_tmp(burger.x_d, y)
    y_intp = get_tmp(burger.y_d, y)

    x_back .= x .- Δt * y
    y_intp .= interp_periodic.(x_back, Ref(y))
    y .= y_intp
    return y
end

function my_init(burger, t)
    u = similar(x)
    @. u = sin(x)
    return u
end

function my_basis_init(burger, t, k)
    ψ = similar(burger.x)
    if k % 2 == 0
        @. ψ = cos(k/2*x)
    else
        @. ψ = sin((k+1)/2*x)
    end
    return ψ
end

function my_step!(burger, u, ustop, tstart, tstop)
    Δt = tstop - tstart
    u = semi_lagrangian(app, u, Δt)
    return
end

function my_step!(burger, u, ustop, tstart, tstop, Ψ)
    Δt = tstop - tstart
    Ψ_new = reduce(hcat, Ψ)
    perturb(r) = semi_lagrangian(burger, u + r'*Ψ, Δt)

    result = DiffResults.DiffResult(u, Ψ_new)
    result = ForwardDiff.jacobian!(result, perturb, zeros(basis_rank))
    for i in size(Ψ)
        Ψ[i] .= Ψ_new[:, i]
    end
end

function my_sum!(burger, a, x, b, y)
    @. y = a*x + b*y
end

function my_access(burger, u)
    println(sum(abs, u))
    # plot!(u)
end

my_norm(burger, u) = LinearAlgebra.norm2(u)
my_innerprod(burger, u, v) = u'*v

my_innerprod (generic function with 1 method)

In [10]:
# test user routines:
test_app = XBraid.BraidApp(
    burger, comm, comm, 
    my_step!, my_init,
    my_sum!, my_norm, my_access,
    my_basis_init, my_innerprod);

In [11]:
XBraid.testInitAccess(test_app, 0.)


Starting braid_TestInitAccess

   braid_TestInitAccess:   Starting Test 1
   braid_TestInitAccess:   u = init(t=0.00e+00)
   braid_TestInitAccess:   access(u) 
10.054678984251698
   braid_TestInitAccess:   check output: wrote u for initial condition at t=0.00e+00. 

   braid_TestInitAccess:   free(u) 
Finished braid_TestInitAccess
Serialized size of user vector: 173
Check output for objects not properly freed:
IdDict{Any, Any}()


In [12]:
XBraid.testSpatialNorm(test_app, 0.)


Starting braid_TestSpatialNorm

   braid_TestSpatialNorm:   Starting Test 1
   braid_TestSpatialNorm:   u = init(t=0.00e+00)
   braid_TestSpatialNorm:   spatialnorm(u) 
   braid_TestSpatialNorm:   v = clone(u)
   braid_TestSpatialNorm:   v = u - v 
   braid_TestSpatialNorm:   spatialnorm(v) 
   braid_TestSpatialNorm:   Test 1 Passed
   braid_TestSpatialNorm:   actual output:    spatialnorm(v) = 0.00e+00  
   braid_TestSpatialNorm:   expected output:  spatialnorm(v) = 0.0 

   braid_TestSpatialNorm:   Starting Test 2
   braid_TestSpatialNorm:   w = clone(u)
   braid_TestSpatialNorm:   w = u + w 
   braid_TestSpatialNorm:   spatialnorm(u)
   braid_TestSpatialNorm:   spatialnorm(w)
   braid_TestSpatialNorm:   Test 2 Passed
   braid_TestSpatialNorm:   actual output:    spatialnorm(w) / spatialnorm(u) = 5.66e+00 / 2.83e+00 = 2.00e+00 
   braid_TestSpatialNorm:   expected output:  spatialnorm(w) / spatialnorm(u) = 2.0 

   braid_TestSpatialNorm:   Starting Test 3
   braid_TestSpatialNorm:  

1

In [13]:
XBraid.testDelta(test_app, 0., 0.1, basis_rank)


Starting braid_TestDelta


Starting braid_TestInnerProd

   braid_TestInnerProd:   Starting Test 1
   braid_TestInnerProd:   u = init_basis(t=0.00e+00, i=0)
   braid_TestInnerProd:   inner_prod(u, u)
   braid_TestInnerProd:   u = u / sqrt(inner_prod(u, u))
   braid_TestInnerProd:   inner_prod(u, u)
   braid_TestInnerProd:   Test 1 Passed
   braid_TestInnerProd:   actual output:    inner_prod(u, u) = 1.00e+00  
   braid_TestInnerProd:   expected output:  inner_prod(u, u) = 1.0 

   braid_TestInnerProd:   Starting Test 2
   braid_TestInnerProd:   v = init(t=0.00e+00, i=1)
   braid_TestInnerProd:   inner_prod(u, v)
   braid_TestInnerProd:   v = v - inner_prod(u, v)*u
   braid_TestInnerProd:   inner_prod(u, v)
   braid_TestInnerProd:   Test 2 Passed
   braid_TestInnerProd:   actual output:    inner_prod(u, v) = 1.11e-16  
   braid_TestInnerProd:   expected output:  inner_prod(u, v) = 0. 

   braid_TestInnerProd:   free(u)
   braid_TestInnerProd:   free(v)

Starting braid_TestInitBasis

  

MethodError: MethodError: no method matching zeros(::typeof(rank))
Closest candidates are:
  zeros(!Matched::Tuple{Vararg{Union{Integer, AbstractUnitRange}}}) at array.jl:585
  zeros(!Matched::Type{StaticArraysCore.MVector{N}}) where N at ~/.julia/packages/StaticArrays/pTgFe/src/MVector.jl:2
  zeros(!Matched::Type{StaticArraysCore.SVector{N}}) where N at ~/.julia/packages/StaticArrays/pTgFe/src/SVector.jl:3
  ...